<a href="https://colab.research.google.com/github/allakoala/data_science/blob/main/colab_notebooks/HW_Classification_(part_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#HW - https://docs.google.com/document/d/1Azj1QEIid13yQNvb3mdwDyfpbgjrP7yhUG5EjKfd_u4/edit

DATA - https://drive.google.com/file/d/1m53GMGvefv99ZeTOuhjb5wLkTSwfMTCf/view

In [ ]:
#basics
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import imblearn
from collections import Counter
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder

from lightgbm import LGBMClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#EDA: exploration of variables and properties of data

#Conclusions:
1. IF on_thyroxine, query_on_thyroxine, on_antithyroid_medication, sick, pregnant, thyroid_surgery, I131_treatment, query_hypothyroid, query_hyperthyroid, lithium, goitre, tumor, hypopituitary, psych, TSH_measured, T3_measured, TT4_measured, T4U_measured, FTI_measured = false,
THEN (FTI,T4U,TSH,T3,TT4) = 0

2. Missing values are imputed:
- sex -  mode
- age -  median

3. The following features were converted to num float values:
- age
- TSH
- T3
- TT4
- T4U
- FTI

3. Outliers except 0 values were substituted with their lower and upper quantile values respectivelyю

7. Target variable:"Class" - is imbalanced
 - negative                   3420
 - compensated_hypothyroid     194
 - primary_hypothyroid          95
 - secondary_hypothyroid         2 - SHOULD BE REMOVED!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#path of the file to read
url = "/content/drive/MyDrive/Colab Notebooks/dataset_57_hypothyroid.csv"

#read the file into a variable
data = pd.read_csv(url, sep=',')

#examine the data for Univariate analysis: consider features separately, their distribution, descriptive statistics, anomalies, omissions, etc
data.head(20)

In [ ]:
#find duplicate rows
duplicate_rows = data.duplicated(subset=data.columns, keep="first")
duplicate_rows.sum()

In [ ]:
#remove duplicate rows
#data = data[~duplicate_rows]
data = data.drop_duplicates()

In [ ]:
#identify the data type
data.info()

In [ ]:
#for each dataset column print unique values
for col in data.columns:
    n_unique_values = data[col].nunique()
    unique_values = data[col].unique()
    print(f"{col}: {n_unique_values}: {unique_values}")

In [ ]:
# look at the "?" value count
data.replace('?', np.nan, inplace=True)
data.isna().sum()

In [ ]:
#missing data for each variable and way to handle it

total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
total_2 = data.isna().sum().sort_values(ascending=False)
percent_2 = (data.isna().sum()/data.isna().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent, total_2, percent_2], axis=1, keys=['Tota_null', 'Percent_null', 'Total_na', 'Percent_na'])
missing_data

In [ ]:
#if TBG_measured, TSH_measured, T3_measured, TT4_measured, T4U_measured, FTI_measured = false, then TBG,FTI,T4U,TSH,T3,TT4 = 0
# replace NaN values with 0 for columns based on their respective measured columns
cols_to_replace = ['TBG', 'FTI', 'T4U', 'TSH', 'T3', 'TT4']
measured_cols = ['TBG_measured', 'FTI_measured', 'T4U_measured', 'TSH_measured', 'T3_measured', 'TT4_measured']
for col, measured_col in zip(cols_to_replace, measured_cols):
    data[col] = data[col].where(data[measured_col] == 't', 0)

data.head(10)

In [ ]:
#missing data for each variable and way to handle it

total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
total_2 = data.isna().sum().sort_values(ascending=False)
percent_2 = (data.isna().sum()/data.isna().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent, total_2, percent_2], axis=1, keys=['Tota_null', 'Percent_null', 'Total_na', 'Percent_na'])
missing_data

In [ ]:
#convert age, TSH, T3, TT4, T4U, FTI float data type
data[['age', 'TSH', 'T3', 'TT4', 'T4U', 'FTI']] = data[['age', 'TSH', 'T3', 'TT4', 'T4U', 'FTI']].astype(float)

In [ ]:
#devide features into num and categirical

#empty lists for numerical and categorical features
numerical_cols = []
categorical_cols = []

#loop over each column and determine its data type
for col in data.columns:
    if data[col].dtype == 'object':
        categorical_cols.append(col)
    else:
        numerical_cols.append(col)

print("All columns:", data.columns)
print("Numerical features:", numerical_cols)
print("Categorical features:", categorical_cols)

In [ ]:
# replace missing values in sex column with mode
data['sex'].fillna(data['sex'].mode()[0], inplace=True)

# replace missing values in age column with median
data['age'].fillna(data['age'].median(), inplace=True)

# print the first 10 rows of the updated dataset
print(data)

In [ ]:
#missing data for each variable and way to handle it

total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
total_2 = data.isna().sum().sort_values(ascending=False)
percent_2 = (data.isna().sum()/data.isna().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent, total_2, percent_2], axis=1, keys=['Tota_null', 'Percent_null', 'Total_na', 'Percent_na'])
missing_data

In [ ]:
data.hist(figsize = (20,20));

In [ ]:
#continuous variables distribution visualization using a histogram

#summary statistics of the numerical features
print(data[numerical_cols].describe())

#the histogram is colored by the target variable 'y' (which is binary) to see the differences in the distribution between the two target classes.
for col in numerical_cols:
    sns.histplot(data=data, x=col, kde=True)
    plt.show()

#lists of columns with specified characteristics
stats = data[numerical_cols].describe()
print(f"\nColumns with mean < median:\n{stats.columns[stats.loc['mean'] < stats.loc['50%']]}")
print(f"\nColumns with mean > median:\n{stats.columns[stats.loc['mean'] > stats.loc['50%']]}")
print(f"\nColumns with big difference between (75th %tile and max) or/and (25th %tile and min values):\n{stats.columns[((stats.loc['75%'] - stats.loc['max']).abs() > 100) | ((stats.loc['25%'] - stats.loc['min']).abs() > 100)]}")
print(f"\nColumns with high standard deviation:\n{stats.columns[stats.loc['std'] > 100]}")
print(f"\nColumns with low standard deviation:\n{stats.columns[stats.loc['std'] < 0.1]}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# create empty lists for positive and negative slopes
positive_slope_pairs = []
negative_slope_pairs = []

for i in range(len(numerical_cols)):
    for j in range(i+1, len(numerical_cols)):
        # calculate the slope of the regression line
        slope = data[numerical_cols[j]].corr(data[numerical_cols[i]])

        # create scatter plot with color-coded points
        sns.scatterplot(x=numerical_cols[i], y=numerical_cols[j], data=data)

        # add regression line
        sns.regplot(x=numerical_cols[i], y=numerical_cols[j], data=data, scatter=False, color="black")

        # set plot title and axis labels
        plt.title(f"{numerical_cols[i]} vs {numerical_cols[j]}")
        plt.xlabel(numerical_cols[i])
        plt.ylabel(numerical_cols[j])

        # append feature pair to the appropriate list based on the sign of the slope
        if slope > 0:
            positive_slope_pairs.append(f"{numerical_cols[i]} vs {numerical_cols[j]}")
        elif slope < 0:
            negative_slope_pairs.append(f"{numerical_cols[i]} vs {numerical_cols[j]}")

        # display plot
        plt.show()

        # clear current figure to free up memory
        plt.clf()

# print out the lists of feature pairs
print("Feature pairs with positive slopes:")
for feature_pair in positive_slope_pairs:
    print(feature_pair)

print("\nFeature pairs with negative slopes:")
for feature_pair in negative_slope_pairs:
    print(feature_pair)

In [ ]:
#heatmap style
sns.set(style='darkgrid')
corrmat = data.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, annot=True, fmt=".2f", cmap='coolwarm', vmax=.8, square=True)
plt.show()

# print highly positively correlated pairs
pos_corr_pairs = []
for i in range(len(corrmat.columns)):
    for j in range(i+1, len(corrmat.columns)):
        if abs(corrmat.iloc[i, j]) >= 0.5:
            pos_corr_pairs.append((corrmat.columns[i], corrmat.columns[j]))

print("Highly positively correlated pairs:")
for pair in pos_corr_pairs:
    print(pair)

# print highly negatively correlated pairs
neg_corr_pairs = []
for i in range(len(corrmat.columns)):
    for j in range(i+1, len(corrmat.columns)):
        if abs(corrmat.iloc[i, j]) <=  -0.5:
            neg_corr_pairs.append((corrmat.columns[i], corrmat.columns[j]))

print("Highly negatively correlated pairs:")
for pair in neg_corr_pairs:
    print(pair)

In [ ]:
#frequency table of the categorical features

for col in categorical_cols:
  print(f"\n{col}:")
  print(data[col].value_counts())

#distribution of the categorical features visualisation using a bar plot
plt.style.use('seaborn-darkgrid')
plt.rcParams.update({'font.size': 12})

for col in categorical_cols:
        plt.figure(figsize=(20,7))
        sns.countplot(data=data, x=col)
        plt.xticks(rotation=90)
        plt.xlabel(col, fontsize=14)
        plt.ylabel('Count', fontsize=14)
        plt.title(f'Distribution of {col}', fontsize=16)
        plt.show()

In [ ]:
#outliers detection - Interquartile Range (IQR) method (values outside the normal range)
for col in numerical_cols:
    vals = data[col].values
    q1, q3 = np.percentile(vals, [25, 75])
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    outliers = vals[(vals < lower) | (vals > upper)]
    unique_vals = data[col][data[col].isin(outliers)].value_counts()
    print(f"{col} has {len(outliers)} outliers with the following unique value counts:\n{unique_vals}\n")

In [ ]:
for col in numerical_cols:
    zero_vals = data.loc[data[col].isin([0, 0.0, 0.00]), col]
    print(f"{col} has {len(zero_vals)} rows with a value of 0, 0.0, or 0.00:\n{zero_vals}\n")

In [ ]:
for outlier in outliers:
    if outlier not in [0, 0.0, 0.00]:
        if outlier < lower:
            data[col][data[col] == outlier] = lower
        else:
            data[col][data[col] == outlier] = upper

In [ ]:
for col in numerical_cols:
    zero_vals = data.loc[data[col].isin([0, 0.0, 0.00]), col]
    print(f"{col} has {len(zero_vals)} rows with a value of 0, 0.0, or 0.00:\n{zero_vals}\n")

In [ ]:
# Categorical encoding for y
labelencoder = LabelEncoder()
data['y'] = labelencoder.fit_transform(data['Class'])
print(data['y'].value_counts())
print(data['y'])

In [ ]:
# Filter out entries where y = 3
data = data[data['y'] != 3]
y=data['y']
print(y.value_counts())
print(y)

In [ ]:
plt.close();
sns.set_style('whitegrid');
sns.pairplot(data, hue='y', height=3);
plt.show()

#Data preprocessing:
1. sex and referral_source are  encoded with dummy data
2. on_thyroxine,  query_on_thyroxine, on_antithyroid_medication, sick, pregnant, thyroid_surgery, I131_treatment, query_hypothyroid, query_hyperthyroid, lithium, goitre, tumor, hypopituitary psych, TSH_measured, T3_measured, TT4_measured, T4U_measured, FTI_measured encode as binary data
4. Lable encoding for y = Class
5. Dropping unimportant features - 'sex', 'referral_source', 'Class', 'TBG','TBG_measured'
6. Data normalization for continues features


In [ ]:
# Create dummy variables for the 'sex' column
sex_dummy = pd.get_dummies(data['sex'], prefix='sex')
sex_dummy

In [ ]:
# Encode binary columns with 1s and 0s
binary_cols = ['on_thyroxine', 'query_on_thyroxine', 'on_antithyroid_medication',
               'sick', 'pregnant', 'thyroid_surgery', 'I131_treatment',
               'query_hypothyroid', 'query_hyperthyroid', 'lithium', 'goitre',
               'tumor', 'hypopituitary', 'TSH_measured', 'T3_measured',
               'TT4_measured', 'T4U_measured', 'FTI_measured', 'psych']
data[binary_cols] = data[binary_cols].apply(lambda x: x.map({'t': 1, 'f': 0}))
data

In [ ]:
# Create dummy variables for the 'referral_source' column
referral_dummy = pd.get_dummies(data['referral_source'], prefix='referral')
referral_dummy

In [ ]:
# Merge the encoded columns with the original data
data_encoded = data.drop(['sex', 'referral_source', 'Class', 'TBG', 'TBG_measured'], axis=1) #'y'
data_encoded = pd.merge(data_encoded, sex_dummy, left_index=True, right_index=True)
data_encoded = pd.merge(data_encoded, referral_dummy, left_index=True, right_index=True)
data_encoded = pd.DataFrame(data=data_encoded, columns=data_encoded.columns)

data_encoded

In [ ]:
#missing data for each variable

total = data_encoded.isnull().sum().sort_values(ascending=False)
percent = (data_encoded.isnull().sum()/data_encoded.isnull().count()).sort_values(ascending=False)

missing_data = pd.concat([total, percent], axis=1, keys=['Tota_null', 'Percent_null'])
missing_data

In [ ]:
# separate binary features from continuous numerical features
binary_features = ['y'] + [col for col in data_encoded.columns if data_encoded[col].nunique() == 2]
numerical_features = [col for col in data_encoded.columns if col not in binary_features]

# print the binary and numerical features
print("Binary Features: ", binary_features)
print("Numerical Features: ", numerical_features)

In [ ]:
# data normalization - Standardization
from sklearn.preprocessing import StandardScaler

scaler_s = StandardScaler().fit(data_encoded[numerical_features])
rescaled_d = scaler_s.transform(data_encoded[numerical_features])
#np.set_printoptions(precision=3)

# create a new dataframe with the rescaled numerical features
rescaled_df = pd.DataFrame(data=rescaled_d, columns=numerical_features)
rescaled_df

In [ ]:
#missing data for each variable

total = rescaled_df.isnull().sum().sort_values(ascending=False)
percent = (rescaled_df.isnull().sum()/rescaled_df.isnull().count()).sort_values(ascending=False)

missing_data = pd.concat([total, percent], axis=1, keys=['Tota_null', 'Percent_null'])
missing_data

In [ ]:
# Merge the binary and numerical features
data_rescaled = data_encoded[binary_features].merge(rescaled_df, how='inner', left_index=True, right_index=True)
#data_rescaled = data_encoded[binary_features].merge(rescaled_df, how='inner', on='index')
#data_rescaled = pd.merge(data_encoded[binary_features], rescaled_df, left_index=True, right_index=True)
print(data_rescaled)

In [ ]:
# Merge the binary and numerical features and convert to numeric - nan values - ???
#data_rescaled = data_encoded[binary_features].join(rescaled_df)
#data_rescaled

In [ ]:
#missing data

total = data_rescaled.isnull().sum().sort_values(ascending=False)
percent = (data_rescaled.isnull().sum()/data_rescaled.isnull().count()).sort_values(ascending=False)

missing_data = pd.concat([total, percent], axis=1, keys=['Tota_null', 'Percent_null'])
missing_data

#Data balancing:
1. SMOTE: Class=1, n=3367 (33.333%)
Class=0, n=3367 (33.333%)
Class=2, n=3367 (33.333%)
2. RandomForestClassifier - Mean Accuracy: 0.922 (0.001)
3. Resample technique: 1:    2533
0:    2533
2:    2533

In [ ]:
print(data_rescaled['y'].value_counts())
y = data_rescaled['y'].values
# y = y.values
print('y:', y.shape)
X = data_rescaled.drop('y', axis=1).values
print('X:', X.shape)

In [ ]:
#from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTE

oversample = SMOTE(k_neighbors=10) #https://machinelearningmastery.com/multi-class-imbalanced-classification/
X_resampled, y_resampled = oversample.fit_resample(X, y) #ValueError: Expected n_neighbors <= n_samples,  but n_samples = 2, n_neighbors = 3

#summarize distribution
counter = Counter(y_resampled)
for k, v in counter.items():
   per = v / len(y_resampled) * 100
   print('Class=%d, n=%d (%.3f%%)' % (k, v, per))

#plot the distribution
pyplot.bar(counter.keys(), counter.values())
pyplot.show()

In [ ]:
# RandomForestClassifier: https://machinelearningmastery.com/multi-class-imbalanced-classification/
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier

# evaluate a model
def evaluate_model(X, y, model):
  # define evaluation procedure
  cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=3, random_state=1)
  # evaluate model
  scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  return scores

# define the model
weights = {0:10.0, 1:1.0, 2:30.0, 3:100.0}
model = RandomForestClassifier(n_estimators=1000, class_weight=weights)
# evaluate the model
scores = evaluate_model(X, y, model)
# summarize performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
from sklearn.utils import resample #https://towardsdatascience.com/methods-for-dealing-with-imbalanced-data-5b761be45a18 Undersample majority class - ?

y = data_rescaled['y']
X = data_rescaled.drop('y', axis=1)

# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

# concatenate our training data back together
X = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
neg = X[X.y==0]
pos_1 = X[X.y==1]
pos_2 = X[X.y==2]

# upsample minority
neg_upsampled = resample(neg,
                          replace=True, # sample with replacement
                          n_samples=len(pos_1), # match number in majority class
                          random_state=27) # reproducible results

pos_2_upsampled = resample(pos_2,
                          replace=True, # sample with replacement
                          n_samples=len(pos_1), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([pos_1, neg_upsampled, pos_2_upsampled,])

# check new class counts
upsampled.y.value_counts()

#X_t, y_t
y_train = upsampled.y
X_train = upsampled.drop('y', axis=1)

#Metrics and cross-validation have been chosen based on target distribution as well as reasoning behind:

1. Considering the algorithm https://machinelearningmastery.com/tour-of-evaluation-metrics-for-imbalanced-classification/: predicting class labels, False Positives More Important: Use F0.5-Measure

3. Precision and recall to focus on small positive class — When the positive class is smaller and the ability to detect correctly positive samples is our main focus (correct detection of negatives examples is less important to the problem) we should use precision and recall.

4.  ROC  - when the positives are the majority or switch the labels and use precision and recall — When the positive class is larger we should probably use the ROC metrics because the precision and recall would reflect mostly the ability of prediction of the positive class and not the negative class which will naturally be harder to detect due to the smaller number of samples. If the negative class (the minority in this case) is more important, we can switch the labels and use precision and recall (labels=[pos_label] - ?). https://towardsdatascience.com/what-metrics-should-we-use-on-imbalanced-data-set-precision-recall-roc-e2e79252aeba


In [ ]:
# Metrics and cross-validation
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.exceptions import ConvergenceWarning
import warnings

# suppress convergence warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# define KNN and Logistic Regression models
knn = KNeighborsClassifier()
lr = LogisticRegression(max_iter=1000)

# K-Fold Cross-Validation
knn_scores = cross_val_score(knn, X_train, y_train, cv=5)
lr_scores = cross_val_score(lr, X_train, y_train, cv=5)

# print results
print("KNN scores: {:.2f} +/- {:.2f}".format(knn_scores.mean(), knn_scores.std()))
print("Logistic Regression scores: {:.2f} +/- {:.2f}".format(lr_scores.mean(), lr_scores.std()))

#Feature importance and hyperparameters tuning: https://scikit-learn.org/stable/modules/classes.html#hyper-parameter-optimizers

1. GridSearchCV with knn: Best score: 0.9415708048924154 Best params: {'n_neighbors': 3}
2. HalvingGridSearchCV with knn: Best score: 0.9415323339587015 Best params: {'n_neighbors': 3}
3. RandomizedSearchCV with knn: Best score: 0.9415708048924154 Best params: {'n_neighbors': 3}
4. HalvingRandomSearchCV with knn: Best score: 0.44565217391304346 Best params: {'n_neighbors': 20}
5. GridSearchCV with logistic: Best score: 0.49533115623159285 Best params: {'C': 0.1, 'penalty': 'l2'}
6. HalvingRandomSearchCV with logistic: best score = 0.4640070921985816, best params = {'penalty': 'l2', 'C': 10}
7. RandomizedSearchCV with logistic: best score = 0.49533115623159285, best params = {'penalty': 'l2', 'C': 0.1}
8. HalvingRandomSearchCV with logistic: best score = 0.4640070921985816, best params = {'penalty': 'l2', 'C': 10}
9. GridSearchCV with lgbm: best score = 0.993946069089775, best params = {'learning_rate': 0.1, 'n_estimators': 300}

10. HalvingGridSearchCV with lgbm: best score = 0.9935356200527705, best params = {'learning_rate': 0.1, 'n_estimators': 300}



11. RandomizedSearchCV with lgbm: best score = 0.993946069089775, best params = {'n_estimators': 300, 'learning_rate': 0.1}


12. HalvingRandomSearchCV with lgbm: best score = 0.7494517543859649, best params = {'n_estimators': 300, 'learning_rate': 0.1}

13.
	Feature importance for lgbm

| Feature                    | Importance |
| --------------------------| ----------|
| age                        | 5100      |
| TSH                        | 4435      |
| TT4                        | 3735      |
| T4U                        | 3658      |
| FTI                        | 3250      |
| T3                         | 2564      |
| sex_F                      | 461       |
| referral_other            | 380       |
| T3_measured                | 343       |
| query_hypothyroid          | 324       |
| referral_SVI               | 280       |
| query_hyperthyroid         | 251       |
| T4U_measured               | 223       |
| thyroid_surgery            | 217       |
| psych                      | 200       |
| referral_SVHC              | 176       |
| on_thyroxine               | 175       |
| referral_STMW              | 171       |
| tumor                      | 167       |
| TT4_measured               | 163       |
| sick                       | 143       |
| I131_treatment             | 114       |
| TSH_measured               | 101       |
| query_on_thyroxine         | 88        |
| sex_M                      | 76        |
| goitre                     | 50        |
| on_antithyroid_medication  | 49        |
| referral_SVHD              | 40        |
| FTI_measured               | 27        |
| pregnant                   | 25        |
| lithium                    | 14        |
| hypopituitary              | 0         |





In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.model_selection import (
    GridSearchCV,
    HalvingGridSearchCV,
    RandomizedSearchCV,
    HalvingRandomSearchCV,
)
import warnings
import numpy as np
import pandas as pd

# Define models
models = {
    "knn": KNeighborsClassifier(),
    "logistic": LogisticRegression(),
    "lgbm": LGBMClassifier()
}

# Define hyperparameters to tune for each model
hyperparameters = {
    "knn": {"n_neighbors": [3, 5, 7, 10, 20, 50]},
    "logistic": {"penalty": ["l1", "l2"], "C": [0.1, 1, 10, 100, 1000]},
    "lgbm": {"learning_rate": [0.01, 0.1, 1, 10, 100, 1000], "n_estimators": [50, 100, 200, 300]}
}

# Define optimizers to use for hyperparameter tuning
optimizers = [
    GridSearchCV,
    HalvingGridSearchCV,
    RandomizedSearchCV,
    HalvingRandomSearchCV
]

# Loop over each model and optimizer, and perform hyperparameter tuning
for model_name, model in models.items():
    for optimizer in optimizers:
        # Create parameter grid for hyperparameter tuning
        param_grid = hyperparameters[model_name]

        # Initialize optimizer and fit to data
        try:
            if optimizer in [HalvingGridSearchCV, HalvingRandomSearchCV]:
                clf = optimizer(model, param_grid, scoring="accuracy", n_jobs=-1, factor=2, cv=5)
            else:
                clf = optimizer(model, param_grid, scoring="accuracy", n_jobs=-1, cv=5)

            clf.fit(X_train, y_train)

            # Print results
            print(f"{optimizer.__name__} with {model_name}: best score = {clf.best_score_}, best params = {clf.best_params_}")

            # Fit model with best hyperparameters and print feature importance (if available)
            best_model = clf.best_estimator_
            best_model.fit(X_train, y_train)
            if hasattr(best_model, 'feature_importances_'):
                feature_importance = pd.DataFrame(list(zip(X.columns, best_model.feature_importances_)), columns=["Feature", "Importance"])
                feature_importance = feature_importance.sort_values("Importance", ascending=False)
                print(f"Feature importance for {model_name}:")
                print(feature_importance)
        except (ValueError, RuntimeError) as e:
            warnings.warn(str(e))

#Models evaluation:

1. K-nearest neighbours https://towardsdatascience.com/multiclass-classification-using-k-nearest-neighbours-ca5281a9ef76
2. Logistic Regression w\wo regularization https://machinelearningmastery.com/multinomial-logistic-regression-with-python/
3. LGBM classifier

#Conclusions:
1. The LGBM classifier performed the best in predicting the target variable, with a high accuracy and good precision, recall, and F1 score for most classes.
2. The K-nearest neighbors model had an accuracy of 0.749, indicating that it correctly predicted the target variable around 75% of the time.
3. The confusion matrix for the K-nearest neighbors model showed that the model performed poorly in predicting classes 0 and 2, with a low precision, recall, and F1 score. The model performed the best in predicting class 1, with a high precision, recall, and F1 score.
4. The logistic regression without regularization model had a very low accuracy of 0.33, indicating poor performance in predicting the target variable.
5. The confusion matrix for the logistic regression without regularization model showed that the model performed very poorly in predicting classes 0 and 2, with a low precision, recall, and F1 score. The model performed better in predicting class 1, with a high precision, recall, and F1 score.
6. The logistic regression with L2 regularization model had a slightly better accuracy of 0.32 than the previous model, but it still performed poorly in predicting the target variable.
7. The confusion matrix for the logistic regression with L2 regularization model showed that the model performed very poorly in predicting classes 0 and 2, with a low precision, recall, and F1 score. The model performed better in predicting class 1, with a high precision, recall, and F1 score.
8. The LGBM classifier had an accuracy of 0.924, indicating good performance in predicting the target variable.
9. The confusion matrix for the LGBM classifier showed that the model performed well in predicting classes 0, 1, and 2, with a high precision, recall, and F1 score. However, the model performed poorly in predicting class 3, which had no samples in the dataset.



In [ ]:
!pip install -U scikit-learn

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score,
                             precision_score, recall_score, f1_score, fbeta_score,
                             roc_curve, roc_auc_score, average_precision_score,
                             log_loss, brier_score_loss)
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# Fit K-nearest neighbours classifier
knn_clf = KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(X_train, y_train)
y_pred_knn_clf = knn_clf.predict(X_test)

# Evaluate performance
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred_knn_clf))
print('Classification Report:')
print(classification_report(y_test, y_pred_knn_clf))
print('Accuracy:', accuracy_score(y_test, y_pred_knn_clf))

try:
  print('Sensitivity:', recall_score(y_test, y_pred_knn_clf, average='weighted'))
except:
  print('Sensitivity: Undefined')

try:
  print('Specificity:', recall_score(y_test, y_pred_knn_clf, average='weighted', pos_label=0))
except:
  print('Specificity: Undefined')

print('G-Mean:', (recall_score(y_test, y_pred_knn_clf, average='weighted') * recall_score(y_test, y_pred_knn_clf, average='weighted')) ** 0.5)

print('Precision:', precision_score(y_test, y_pred_knn_clf, average='weighted', labels=[0, 1, 2, 3]))
print('Recall:', recall_score(y_test, y_pred_knn_clf, average='weighted', labels=[0, 1, 2, 3]))
print('F1 Score:', f1_score(y_test, y_pred_knn_clf, average='weighted', labels=[0, 1, 2, 3]))
print('F0.5 Score:', fbeta_score(y_test, y_pred_knn_clf, beta=0.5, average='weighted', labels=[0, 1, 2, 3]))
print('F1 Score (weighted):', f1_score(y_test, y_pred_knn_clf, average='weighted', labels=[0, 1, 2, 3]))

# # Compute ROC curve and AUC score - doesnt work for multiclass - ?
# y_prob_knn_clf = knn_clf.predict_proba(X_test)[:, 1]
# fpr, tpr, thresholds = roc_curve(y_test, y_prob_knn_clf)
# print('ROC Curve:')
# print('False Positive Rates:', fpr)
# print('True Positive Rates:', tpr)
# print('ROC AUC Score:', roc_auc_score(y_test, y_prob_knn_clf))

# # Compute PR curve and AUC score
# precision, recall, thresholds = precision_recall_curve(y_test, y_prob_knn_clf)
# print('PR Curve:')
# print('Precision:', precision)
# print('Recall:', recall)
# print('PR AUC Score:', average_precision_score(y_test, y_prob_knn_clf))

# # Compute log loss and Brier score
# print('Log Loss:', log_loss(y_test, y_prob_knn_clf))
# print('Brier Score:', brier_score_loss(y_test, y_prob_knn_clf))
# print('Brier Skill Score:', brier_score_loss(y_test, y_prob_knn_clf) - brier_score_loss(y_test, y_test))

In [ ]:
# Logistic Regression model without regularization https://ml-cheatsheet.readthedocs.io/en/latest/logistic_regression.html - ?
import warnings
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, average_precision_score, brier_score_loss, classification_report, confusion_matrix, f1_score, fbeta_score, log_loss, precision_score, recall_score, roc_auc_score, roc_curve)

#Ignore undefined metric warnings
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

#Create and fit logistic regression model without regularization
lr_clf = LogisticRegression(penalty='none')
lr_clf.fit(X_train, y_train)

#Make predictions on test data
y_pred_lr = lr_clf.predict(X_test)

#Print model performance metrics
print('Logistic Regression without regularization model')
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred_lr))
print('Classification Report:')
print(classification_report(y_test, y_pred_lr))
print('Accuracy:', accuracy_score(y_test, y_pred_lr))

#Calculate performance metrics
try:
  print('Sensitivity:', recall_score(y_test, y_pred_lr, average='weighted'))
except UndefinedMetricWarning:
  print('Sensitivity: Undefined')

try:
  print('Specificity:', recall_score(y_test, y_pred_lr, average='weighted', pos_label=0)) #
except UndefinedMetricWarning:
  print('Specificity: Undefined')

print('G-Mean:', (recall_score(y_test, y_pred_lr, average='weighted') * recall_score(y_test, y_pred_lr, average='weighted')) ** 0.5)

print('Precision:', precision_score(y_test, y_pred_lr, average='weighted', labels=[0, 1, 2, 3]))
print('Recall:', recall_score(y_test, y_pred_lr, average='weighted', labels=[0, 1, 2, 3]))
print('F-Measure:', f1_score(y_test, y_pred_lr, average='weighted', labels=[0, 1, 2, 3]))
print('F0.5-Measure:', fbeta_score(y_test, y_pred_lr, beta=0.5, average='weighted', labels=[0, 1, 2, 3]))
print('F1-Measure:', fbeta_score(y_test, y_pred_lr, beta=1, average='weighted'))

# #Calculate and print ROC curve and AUC  - doesnt work for multiclass - ?
# fpr, tpr, _ = roc_curve(y_test, y_pred_lr)
# print('ROC curve:', fpr, tpr)
# print('True Positive Rate:', tpr)
# print('False Positive Rate:', fpr)
# print('ROC AUC:', roc_auc_score(y_test, y_pred_lr))

# #Calculate and print PR AUC and log loss
# print('PR AUC:', average_precision_score(y_test, y_pred_lr, average='weighted'))
# print('Log Loss:', log_loss(y_test, y_pred_lr))

# #Calculate and print Brier score loss and skill score
# print('Brier Score Loss:', brier_score_loss(y_test, y_pred_lr))
# print('Brier Skill Score:', brier_score_loss(y_test, y_pred_lr) - brier_score_loss(y_test, y_test.mean()))

In [ ]:
# Logistic Regression model with L2 regularization
lr_l2 = LogisticRegression(C=100, penalty='l2')
lr_l2.fit(X_train, y_train)
y_pred_lr_l2 = lr_l2.predict(X_test)

# Print confusion matrix and classification report
print('Logistic Regression with L2 regularization')
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred_lr_l2))
print('Classification Report:')
print(classification_report(y_test, y_pred_lr_l2))

# Calculate and print evaluation metrics
print('Accuracy:', accuracy_score(y_test, y_pred_lr_l2))

print('G-Mean:', (recall_score(y_test, y_pred_lr_l2, average='weighted') * recall_score(y_test, y_pred_lr_l2, average='weighted')) ** 0.5)

print('Precision:', precision_score(y_test, y_pred_lr_l2, average='weighted', labels=[0, 1, 2, 3]))
print('Recall:', recall_score(y_test, y_pred_lr_l2, average='weighted', labels=[0, 1, 2, 3]))
print('F-Measure:', f1_score(y_test, y_pred_lr_l2, average='weighted', labels=[0, 1, 2, 3]))
print('F0.5-Measure:', fbeta_score(y_test, y_pred_lr_l2, beta=0.5, average='weighted', labels=[0, 1, 2, 3]))
print('F1-Measure:', fbeta_score(y_test, y_pred_lr_l2, beta=1, average='weighted', labels=[0, 1, 2, 3]))

#Set zero_division parameter to avoid the "UndefinedMetricWarning"
print('Sensitivity:', recall_score(y_test, y_pred_lr_l2, zero_division=1, average='weighted'))
print('Specificity:', recall_score(y_test, y_pred_lr_l2, pos_label=0, zero_division=1, average='weighted'))

# fpr, tpr, _ = roc_curve(y_test, y_pred_lr_l2) #doesnt work
# print('ROC curve:', fpr, tpr)
# print('True Positive Rate:', tpr)
# print('False Positive Rate:', fpr)
# print('ROC AUC:', roc_auc_score(y_test, y_pred_lr_l2))
# print('PR AUC:', average_precision_score(y_test, y_pred_lr_l2, average='weighted'))
# print('Log Loss:', log_loss(y_test, y_pred_lr_l2))
# print('Brier Score Loss:', brier_score_loss(y_test, y_pred_lr_l2))
# print('Brier Skill Score:', brier_score_loss(y_test, y_pred_lr_l2) - brier_score_loss(y_test, y_test.mean()))

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score,
                             precision_score, recall_score, f1_score, fbeta_score)

# Fit LGBM classifier
lgbm_clf = LGBMClassifier(learning_rate=0.1, n_estimators=300)
lgbm_clf.fit(X_train, y_train)
y_pred_lgbm_clf = lgbm_clf.predict(X_test)

# Evaluate performance
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred_lgbm_clf))
print('Classification Report:')
print(classification_report(y_test, y_pred_lgbm_clf))
print('Accuracy:', accuracy_score(y_test, y_pred_lgbm_clf))

try:
  print('Sensitivity:', recall_score(y_test, y_pred_lgbm_clf, average='weighted'))
except:
  print('Sensitivity: Undefined')

try:
  print('Specificity:', recall_score(y_test, y_pred_lgbm_clf, average='weighted', pos_label=0))
except:
  print('Specificity: Undefined')

print('G-Mean:', (recall_score(y_test, y_pred_lgbm_clf, average='weighted') * recall_score(y_test, y_pred_lgbm_clf, average='weighted')) ** 0.5)

print('Precision:', precision_score(y_test, y_pred_lgbm_clf, average='weighted'))
print('Recall:', recall_score(y_test, y_pred_lgbm_clf, average='weighted'))
print('F1 Score:', f1_score(y_test, y_pred_lgbm_clf, average='weighted'))
print('F0.5 Score:', fbeta_score(y_test, y_pred_lgbm_clf, beta=0.5, average='weighted'))
print('F1 Score (weighted):', f1_score(y_test, y_pred_lgbm_clf, average='weighted'))

# Compute ROC curve and AUC score
# y_prob_lgbm_clf = lgbm_clf.predict_proba(X_test)[:, 1]
# fpr, tpr, thresholds = roc_curve(y_test, y_prob_lgbm_clf)
# print('ROC Curve:')
# print('False Positive Rates:', fpr)
# print('True Positive Rates:', tpr)
# print('ROC AUC Score:', roc_auc_score(y_test, y_prob_lgbm_clf))

# # Compute PR curve and AUC score
# precision, recall, thresholds = precision_recall_curve(y_test, y_prob_lgbm_clf)
# print('PR Curve:')
# print('Precision:', precision)
# print('Recall:', recall)
# print('PR AUC Score:', average_precision_score(y_test, y_prob_lgbm_clf))

# # Compute log loss and Brier score
# print('Log Loss:', log_loss(y_test, y_prob_lgbm_clf))
# print('Brier Score:', brier_score_loss(y_test, y_prob_lgbm_clf))
# print('Brier Skill Score:', brier_score_loss(y_test, y_prob_lgbm_clf) - brier_score_loss(y_test, y_test))

Popular algorithms that can be used for multi-class classification include:
 - Decision Trees. https://towardsdatascience.com/decision-trees-in-machine-learning-641b9c4e8052 https://blog.paperspace.com/decision-trees/ https://www.geeksforgeeks.org/decision-tree-introduction-example/
 - Naive Bayes.
 - Random Forest.
 - Gradient Boosting. https://mlcourse.ai/book/topic10/topic10_intro.html

Binary classification algorithms that can use these strategies for multi-class classification include:
 - Logistic Regression.
 - Support Vector Machine.

Linear discriminant analysis - https://en.wikipedia.org/wiki/Linear_discriminant_analysis https://scikit-learn.org/stable/modules/lda_qda.html

labels=[pos_label] - ?

from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

# create a list of models for voting ensemble
ensemble_models = [('rfc', rfc_best), ('knn', knn_best), ('lr', lr_best), ('lgbm', lgbm_best), ('dt', dt_best), ('gb', gb_best)]

# create a voting classifier with soft voting
voting_clf_soft = VotingClassifier(estimators=ensemble_models, voting='soft')

# fit the voting classifier
voting_clf_soft.fit(X_train, y_train)

# evaluate the voting classifier
y_pred_soft = voting_clf_soft.predict(X_test)
accuracy_soft = accuracy_score(y_test, y_pred_soft)
print("Soft voting accuracy:", accuracy_soft)

# voting classifier with hard voting
voting_clf_hard = VotingClassifier(estimators=ensemble_models, voting='hard')

# fit the voting classifier
voting_clf_hard.fit(X_train, y_train)

# evaluate the voting classifier
y_pred_hard = voting_clf_hard.predict(X_test)
accuracy_hard = accuracy_score(y_test, y_pred_hard)
print("Hard voting accuracy:", accuracy_hard)

<!-- :

  a. K-nearest neighbours classifier:
    - Confusion Matrix:
 [ 13  32   6]
 [ 87 719  28]
 [  0  25   2]
    - Classification Report:
              precision    recall  f1-score   support

           0       0.13      0.25      0.17        51
           1       0.93      0.86      0.89       834
           2       0.06      0.07      0.06        27
          accuracy                           0.80       912
          macro avg       0.37      0.40      0.38       912
          weighted avg       0.86      0.80      0.83       912

    - Accuracy: 0.8048245614035088
    - Sensitivity: 0.8048245614035088
    - Specificity: 0.8048245614035088
    - G-Mean: 0.8048245614035088
    - Precision: 0.8562167661421595
    - Recall: 0.8048245614035088
    - F1 Score: 0.8282868487082953
    - F0.5 Score: 0.8446138570242653
    - F1 Score (weighted): 0.8282868487082953


  b. Logistic Regression without regularization model
  - Confusion Matrix:
    [ 29   7  15]
    [220 354 260]
    [ 10   0  17]
  - Classification Report:
              precision    recall  f1-score   support

           0       0.11      0.57      0.19        51
           1       0.98      0.42      0.59       834
           2       0.06      0.63      0.11        27
           accuracy                           0.44       912
           macro avg       0.38      0.54      0.30       91
           weighted avg       0.90      0.44      0.56       912

  - Accuracy: 0.43859649122807015
  - Sensitivity: 0.43859649122807015
  - Specificity: 0.43859649122807015
  - G-Mean: 0.43859649122807015
  - Precision: 0.9047265321998786
  - Recall: 0.43859649122807015
  - F-Measure: 0.5554150294763841
  - F0.5-Measure: 0.7201090277447315
  - F1-Measure: 0.5554150294763841



  c. Logistic Regression with L2 regularization
  - Confusion Matrix:
[[ 28   8  15]
 [221 350 263]
 [  9   1  17]]
  - Classification Report:
              precision    recall  f1-score   support

           0       0.11      0.55      0.18        51
           1       0.97      0.42      0.59       834
           2       0.06      0.63      0.11        27
           accuracy                           0.43       912
           macro avg       0.38      0.53      0.29       912
           weighted avg       0.90      0.43      0.55       912

  - Accuracy: 0.4331140350877193
  - G-Mean: 0.4331140350877193
  - Precision: 0.8993231775547051
  - Recall: 0.4331140350877193
  - F-Measure: 0.5498335728495531
  - F0.5-Measure: 0.7143045483386601
  - F1-Measure: 0.5498335728495531
  - Sensitivity: 0.4331140350877193
  - Specificity: 0.4331140350877193

d. LGBM classifier
  - Confusion Matrix:
[[  1  50   0]
 [  4 829   1]
 [  0  26   1]]
  - Classification Report:
              precision    recall  f1-score   support

           0       0.20      0.02      0.04        51
           1       0.92      0.99      0.95       834
           2       0.50      0.04      0.07        27
           accuracy                           0.91       912
           macro avg       0.54      0.35      0.35       912
           weighted avg       0.86      0.91      0.88       912

  - Accuracy: 0.9111842105263158
  - Sensitivity: 0.9111842105263158
  - Specificity: 0.9111842105263158
  - G-Mean: 0.9111842105263158
  - Precision: 0.8636649462052922
  - Recall: 0.9111842105263158
  - F1 Score: 0.8759178004958162
  - F0.5 Score: 0.8591987280428726
  - F1 Score (weighted): 0.8759178004958162 -->